# Using statistical methods with messy data

For this activity we will look at a very real example of how one of my previous students Sade and I bring together all sorts of exploratory data analysis and tons of different kinds of Python packages - from mathematical packages like `numpy` and `scipy` to data analysis and visualization tools like `pandas` and `seaborn` to geospaital tools in `geopandas`. 

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

import geopandas as gpd
import seaborn as sns

import scipy as sp
from scipy import stats

You may remember creating a shapefile with CALM data (`CALM_points.shp`) - let's put it to use here. 

# How is the active layer changing every year?

So now we have measurements of the active layer thickness (ALT) for each site, but we're going to have to get a little clever here if we want to examine the **trend** in ALT. Data-management-wise, each site has its own unique dataset issues - some sites are missing most years, some sites have data gaps. We need to:
- (1) first clean up the data prior to any trend-finding
- (2) once the data are clean, we want to find the **trend** in ALT

I am going to show you how to get a **linear regression** to the available data:

## Here is an algorithm that grabs an x and y array for years and measurements

In [55]:
data = gpd.read_file("../1007/CALM_points.shp")


In [56]:
year_names = np.arange(1990, 2016, 1)

for sites in range(len(data)): # for all the rows in our dataset

  y_floats = np.array(data.iloc[sites, 6:32].values, dtype=float) # read in the data from columns that are yearly ALT measurements as floats

  y = y_floats[~np.isnan(y_floats)] # find the indices for which that year's measurement is NOT a nan

  x = year_names[~np.isnan(y_floats)] # find the corresponding years for valid data 

  if np.sum([np.isnan(y)==False])>10: # if there are at least 10 valid measurements for the time period

    data.loc[(sites, "average")] = np.mean(y) # grab the mean of those measurements

    res = stats.linregress(x,y) # and regress the year array against the measurement array 
    # res is the result of the linregress function and spits out a list of important numbers

    data.loc[(sites, "slope")] = res[0] # ...and store all that data in our data frame
    data.loc[(sites, "intercept")] = res[1]
    data.loc[(sites, "rvalue")] = res[2]
    data.loc[(sites, "pvalue")] = res[3]
    data.loc[(sites, "stderr")] = res[4]
  else: # if we don't have enough valid data, go to the next row
    continue

Now what do we have?

In [ ]:
data.head()

## Your turn!

(1) Describe in a paragraph (3-4 sentences) **what** the above code does and **why** we have done this (i.e. what does a linear regression do to help answer the question of how ALT is changing over time).

*Your text here*

(2) Please make a data visualization using the `slope` column as a **dependent variable** in the resulting dataframe and `Latitude` as the **independent variable**. You can choose any plotting package and style ou wish, but the visualization should help answer the question "How has active layer thickness changed over the study period, and what factors affect that change?". **Be sure to include axis labels *with proper units*!**

In [58]:
# Your code here

Let's see what this looks like

In [ ]:
# GeoPandas has a simple map of the Earth built in
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

fig, ax = plt.subplots(figsize=(5,5),dpi=300)
im = world.plot(
    color='white', edgecolor='black', ax=ax)

result = data.plot(
        ax=ax,
        column='slope',
        vmin=-2,
        vmax=2,
        cmap='seismic',
        s=10, #size of point
        legend=True,
        legend_kwds={
            'label': "what's a good colorbar label?",
            'orientation': "horizontal"
            }
        )

ax.set_ylim(40,90)

ax.set_ylabel("ylabel")
ax.set_xlabel("xlabel")

I will leave [axis labeling](https://matplotlib.org/stable/api/axes_api.html) to you :)

# Spatial join with permafrost data

We will perform a [spatial join](https://geopandas.org/en/stable/gallery/spatial_joins.html) to determine within which permafrost category each of these data points fall. Permafrost extent tells you the general temperature of the area: continuous permafrost is in the coldest parts of the world, followed by discontinuous, sporadic, and isolated.

In [60]:
permaice = gpd.read_file("permaice_simple.shp")

joined_data = data.to_crs(permaice.crs).sjoin(permaice, how="inner") 

Let's check out the result:

In [ ]:
joined_data

# Visualize and analyze results

I asked Sade, "Ok, so is continuous permafrost thawing faster or slower than discontinuous permafrost?" Let's see what she came up with:

In [62]:
# Make individual data frames for each extent category - it's just a little easier this way
df_cont=joined_data.loc[joined_data['EXTENT']=='C']
df_discont=joined_data.loc[joined_data['EXTENT']=='D']
df_iso=joined_data.loc[joined_data['EXTENT']=='I']
df_spor=joined_data.loc[joined_data['EXTENT']=='S']

# And make them a list
df_extent_list=[df_cont,df_discont,df_iso,df_spor]

Next, she wrote a function to plot each type of permafrost in one of the four plot boxes

In [63]:
def plot_site_timeseries(df, axis):
  for sites in range(len(df)):
    y_floats = np.array(df.iloc[sites, 6:32].values, dtype=float)
    y = y_floats[~np.isnan(y_floats)]
    x = year_names[~np.isnan(y_floats)]
    if np.sum([np.isnan(y)==False])>10:
      fig=sns.regplot(x=x,y=y, ax=axis, 
                      color= color_dict[df.iloc[sites]['EXTENT']], scatter_kws={'alpha':0.5, "s":.25}, line_kws={"lw":.25})
    else:
      continue
      # fig.set(xlabel='Year', ylabel='Active Layer Thickness', title ='Change in ALT from 1991-2015', ylim=(0, 200)

And then ran that function on each category for a figure!

In [ ]:
# This is a color dictionary that tells the function which color to make which category
color_dict = {'C':'r', 'D':'g', 'I':'b' , 'S':'k', 'no_data':'grey'}

fig, ax = plt.subplots(2,2, dpi=150)

# Each category got a call and a designated axis
plot_site_timeseries(df_extent_list[0], ax[0,0])
plot_site_timeseries(df_extent_list[1], ax[0,1])
plot_site_timeseries(df_extent_list[2], ax[1,0])
plot_site_timeseries(df_extent_list[3], ax[1,1])

# These are setting labels
ax[0,0].set_title('Change in Active Layer Thickness in Continuous Permafrost', fontsize=6)
ax[0,0].set_ylabel('Active Layer Thickness in cm',fontsize=6)
ax[0,0].set_xlabel('Year',fontsize=6)
ax[0,0].tick_params(axis='both', which='major', labelsize=5)
ax[0,1].set_title('Change in Active Layer Thickness in Discontinuous Permafrost', fontsize=6)
ax[0,1].set_ylabel('Active Layer Thickness in cm',fontsize=6)
ax[0,1].set_xlabel('Year',fontsize=6)
ax[0,1].tick_params(axis='both', which='major', labelsize=5)
ax[1,0].set_title('Change in Active Layer Thickness in Isolated Permafrost', fontsize=6)
ax[1,0].set_ylabel('Active Layer Thickness in cm',fontsize=6)
ax[1,0].set_xlabel('Year',fontsize=6)
ax[1,0].tick_params(axis='both', which='major', labelsize=5)
ax[1,1].set_title('Change in Active Layer Thickness in Spororatic Permafrost', fontsize=6)
ax[1,1].set_ylabel('Active Layer Thickness in cm',fontsize=6)
ax[1,1].set_xlabel('Year',fontsize=6)
ax[1,1].tick_params(axis='both', which='major', labelsize=5)
plt.tight_layout()

# Your turn!

I would like you to use **one visualization** that tries to answer the question: "Are warmer or colder places thawing more rapidly?"  Be sure to use **proper axis labels** for your data. Help your argument by performing a statistical test like a [t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html).

In [65]:
# Your code here

*Your answer to my question in 2-4 sentences here*

# Or!

Perform a regression on your own data and/or perform at least one statistical measurement that makes sense with your data. Include a 2-4 sentence explanation for <b>what</b> the data represent, <b>why</b> you chose the statistical analysis, and <b>what</b> the interpretation of your statistics tell you about your system. <b>Please be sure to include the data if it lives in a separate file with your submission!</b>

# Deliverables:
- All paragraph answers provided
- All extra plots provided
- If applicable, your dataset and your analyses. 